In [1]:
from stormvogel import model, pgc
from stormvogel.show import show

We can also build a model using the pgc (python guarded command) model builder. Here, we use guards to specify the state we will end up in for each case. This is similar to the structure in prism files.

In [11]:
import math

#changing this constant changes the size of the model
N = 2

#we can use simpler versions of the Action and State class, included specifically for this method of model building
initial_state = pgc.State(x=math.floor(N / 2))
left = pgc.Action(["left"])
right = pgc.Action(["right"])

#this available actions function will specify what actions are available in each state
def available_actions(s: pgc.State):
    if s.x == N:
        return [right]
    elif s.x == 0:
        return [left]
    else :
        return [left,right]

#finally we make the delta function, which says to which states we can transition depending on in which state we are and which action we choose.
p = 0.5
def delta(s: pgc.State, action: pgc.Action):
    if action == left:
        return (
            [
                (p, pgc.State(x=s.x + 1)),
                (1 - p, pgc.State(x=s.x)),
            ]
            if s.x < N
            else []
        )
    elif action == right:
        return (
            [
                (p, pgc.State(x=s.x - 1)),
                (1 - p, pgc.State(x=s.x)),
            ]
            if s.x > 0
            else []
        )

we can also optionally provide functions that assign rewards and labels

In [12]:
def rewards(s: pgc.State, a: pgc.Action):
    return [1, 2]

def labels(s: pgc.State):
    return [str(s.x)]

We then combine all of the above to call the build_pgc function that will build our model using the functions

In [13]:
pgc_model = pgc.build_pgc(
    delta=delta,
    available_actions=available_actions,
    initial_state_pgc=initial_state,
    labels=labels,
    rewards=rewards,
)

print(pgc_model)
show(pgc_model)

ModelType.MDP with name None

States:
State 0 with labels ['init', '1'] and features {}
State 1 with labels ['2'] and features {}
State 2 with labels ['0'] and features {}

Transitions:
Action with labels frozenset({'left'}) => 0.5 -> State 1 with labels ['2'] and features {}, 0.5 -> State 0 with labels ['init', '1'] and features {}; Action with labels frozenset({'right'}) => 0.5 -> State 2 with labels ['0'] and features {}, 0.5 -> State 0 with labels ['init', '1'] and features {}
Action with labels frozenset({'right'}) => 0.5 -> State 0 with labels ['init', '1'] and features {}, 0.5 -> State 1 with labels ['2'] and features {}
Action with labels frozenset({'left'}) => 0.5 -> State 0 with labels ['init', '1'] and features {}, 0.5 -> State 2 with labels ['0'] and features {}


Output()

Output()

State 0 with labels ['init', '1'] and features {} Action with labels frozenset()


RuntimeError: This action is not available in this state

We don't have to use the provided State class, in fact we can use any object we like! Here is an example where we use integers instead of states objects.

In [5]:
def delta(state):
    return [
        (0.5, (state + 1) % 5),
        (0.5, (state - 1) % 5),
    ]
    
def rewards(state):
    return [state]

pgc_model = pgc.build_pgc(delta, initial_state_pgc=0, rewards=rewards, modeltype=model.ModelType.DTMC)
show(pgc_model)

Output()

<IPython.core.display.Javascript object>

Output()